In [1]:
import json
import rdflib
from uuid import uuid4 as uuid
import json
import pandas as pd
import urllib

INFO:rdflib:RDFLib Version: 4.2.1


In [2]:
from rdflib import Namespace, Literal, URIRef, BNode
from rdflib import RDF, RDFS, OWL, XSD

WS = Namespace("http://metroinsight.io/schema/workspace#")
CORE = Namespace("http://metroinsight.io/schema/core#")
BRICK = Namespace("http://buildsys.org/ontologies/Brick#")
BF = Namespace("http://buildsys.org/ontologies/BrickFrame#")
GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
SSN = Namespace("http://www.w3.org/ns/ssn/")
UNIT = Namespace("http://qudt.org/vocab/unit/")
QUDT = Namespace("http://qudt.org/schema/qudt/")
SCHEMAORG = Namespace("https://schema.org/")

##Core scheme

* ssn:Sensor: generated randomly
* ssn:Sensor ssn:hasOutput
* ssn:Sensor geo:location geo:Point
* core:DataStream: dataset
* core:DataStream core:hasDataReference LINK?
* core:DataStream: qudt:unit qudt:Unit

In [3]:
with open("config/bd3config.json", "r") as fp:
    configs = json.load(fp)
    valid_naes = configs['valid_naes']
with open("metadata/bacnet_devices.json", "r") as fp:
    bacnet_devices = json.load(fp)
with open("metadata/building_info.json", "r") as fp:
    building_info_dict = json.load(fp)
with open("metadata/bacnetunit_to_qudt_unit.json", "r") as fp:
    bacnet_unit_map = json.load(fp)

In [20]:
def make_sensor(g, data_ref, sensor_type_str, sensor_label_str='', unit_str=None, latitude=None, longitude=None, altitude=None):
    #Init sensor
    sensor = WS[urllib.quote(str(uuid()), safe='')]
    g.add((sensor, RDF.type, SSN.Sensor))
    sensor_label = Literal(urllib.quote(sensor_label_str, safe=''), datatype=XSD.string)
    g.add((sensor, RDFS.label, sensor_label))
    
    # Property (SSN Style)
    prop = WS[urllib.quote(sensor_type_str, safe='')] # TODO: <- This needs to be standardized.
    g.add((prop, RDF.type, SSN.Property))
    g.add((sensor, SSN.observes, prop))
    
    # Point location
    if latitude and longitude:
        loc = BNode()
        g.add((loc, RDF.type, GEO.Point))
        g.add((loc, GEO.long,  Literal(float(longitude), datatype=XSD.float)))
        g.add((loc, GEO.lat,  Literal(float(latitude), datatype=XSD.float)))
        if altitude:
            g.add((loc, GEO.alt,  Literal(float(altitude), datatype=XSD.float)))
        g.add((sensor, SCHEMAORG.location, loc))
    
    # Data Stream
    #sensor_data = WS[urllib.quote(str(uuid()), safe='')]
    sensor_data = BNode()
    g.add((sensor_data, RDF.type, CORE.DataStream))
    g.add((sensor, SSN.hasOutput, sensor_data))
    
    # Data Reference
    g.add((sensor_data, CORE.hasDataReference, Literal(str(data_ref), datatype=XSD.string)))
    
    # Data Unit
    if unit_str:
        g.add((sensor_data, QUDT.unit, UNIT[unit_str]))

In [21]:
unitSet = set()
g = rdflib.Graph()
g.bind('ws', WS)
g.bind('core', CORE)
g.bind('brick', BRICK)
g.bind('bf', BF)
g.bind('geo', GEO)
g.bind('ssn', SSN)
g.bind('unit', UNIT)
g.bind('qudt', QUDT)
g.bind('schema', SCHEMAORG)

for nae_num in valid_naes[0:1]:
    objs = bacnet_devices[nae_num]['objs']
    location = None
    for building_name, building in building_info_dict.items():
        if int(nae_num) in building['naes']:
            location = building['location']
            latitude = location['latitude']
            longitude = location['longitude']
            break
    if location==None:
        print str(nae_num)
        print "nae is not found in building_info_dict" 
    for obj in objs:
        if obj['data_type'] not in [0,1,2,3,4,5,13,14,19]:
            continue 
        srcid = str(nae_num) + "_" + str(obj['data_type']) + "_" + str(obj['props']['instance'])
        bacnet_unit = str(obj['unit'])
        qudt_unit = bacnet_unit_map.get(bacnet_unit)
        data_ref = 'raw_data/buildingdepot/data/%s.csv'%srcid
        sensor_type_str = obj['jci_name']
        make_sensor(g, data_ref, sensor_type_str, unit_str=qudt_unit, latitude=latitude, longitude=longitude, altitude=None)
        

In [23]:
g.serialize('bd_metro.ttl', format='turtle')

In [6]:
bacnet_devices['505'].keys()

[u'objs',
 u'name',
 u'device_desc_prop',
 u'jci_name',
 u'obj_count',
 u'props',
 u'device_id',
 u'desc']